# Simulate a user accessing scenes to estimate # of sectors needed to unseal

## Imports

In [2]:
%matplotlib inline
import random
import geopandas as gpd
from scipy.spatial import KDTree
import numpy as np

## Prepare Landsat scenes (Rows/Paths) of the US

### Fetch files

In [3]:
# From https://www.usgs.gov/media/files/landsat-wrs-2-descending-path-row-shapefile
landsat_scenes_shp = gpd.read_file("https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip")[["PATH", "ROW", "geometry"]]

# From https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
us_states_shp = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip")[["geometry"]]

# From https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
msa_boundaries = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_cbsa_20m.zip")[["CBSAFP", "geometry"]]

### Reproject to WGS84 (to match Landsat)

In [35]:
us_states_shp = us_states_shp.to_crs(epsg=4326)
msa_boundaries = msa_boundaries.to_crs(epsg=4326)

### Clip Scenes to Continential US 

In [36]:
us_states_continential = us_states_shp[us_states_shp["STUSPS"].isin(["AK", "HI", "PR", "VI", "GU", "AS", "MP", "UM"]) == False]
landsat_scenes_clipped = gpd.clip(landsat_scenes_shp, us_states_continential)
print(len(landsat_scenes_clipped)) # Should be about 457 scenes

457


/Users/jsolly.admin/Documents/code/easier/venv/lib/python3.10/site-packages/geopandas/tools/clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


## Intersect MSAs with Landsat scenes

In [38]:
msa_boundaries_intersect= gpd.sjoin(msa_boundaries, landsat_scenes_clipped, how="inner", op="intersects")

/Users/jsolly.admin/Documents/code/easier/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3318: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


## Drop Geometry of MSAs

In [39]:
msa_boundaries_intersect_drop_geom = msa_boundaries_intersect[["CBSAFP", "PATH", "ROW"]]
msa_boundaries_intersect_drop_geom

## User Settings

In [41]:
scenes_requested_per_request = random.randint(1, 5)
number_of_requests = 1000

## Sector and Cache Settings

In [42]:
cache_time = 1000 # Number of requests (loops) before hot storage cache is cleared
cache_hit_count = 0
sector_scene_count = 25
sector_packing_method = "random-greedy" # "random-greedy" or "sequential"

## Pack Sectors

In [50]:
scenes = landsat_scenes_clipped[["PATH", "ROW"]].values # Only need the path and row to identify a scene
tree = KDTree(scenes)

car_scenes = [] # Each car will have a list of scenes that will be used to pack it. len(car_scenes) = scenes / 25
scene = random.choice(scenes)

while True: # Keep packing as long as there are still scenes left
    k_nearest_neighbors = tree.query(scene, k=25) # Pack each CAR file with 25 scenes
    try:
        scenes_to_pack = [scenes[i] for i in k_nearest_neighbors[1]]
        car_scenes.append(scenes_to_pack)
    except IndexError:
        car_scenes.append(scenes) # If there are less than 25 scenes left, just pack the rest
        break
    scenes = [v for i, v in enumerate(scenes) if i not in k_nearest_neighbors[1]] # Remove the scenes that were packed into the CAR file
    tree = KDTree(scenes) # Rebuild tree with nearest neighbors removed
    scene = random.choice(scenes)

In [1]:
# plt.plot(landsat_scenes_clipped)

NameError: name 'plt' is not defined

## Simulate User Access

## Plot Results